# Gradient Descent

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("E:\Data Science\datasets\insurance_data.csv")

In [3]:
df.head()

,age,bought_insurance
0,22,0
1,25,0
2,47,1
3,52,0
4,46,1


In [4]:
df.shape

(27, 2)

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['age'], df['bought_insurance'], test_size=0.1, random_state=43)

In [6]:
X_train_scaled = X_train.copy()
X_train_scaled = X_train_scaled/100

X_test_scaled = X_test.copy()
X_test_scaled = X_test_scaled/100

In [7]:
X_train_scaled, X_test_scaled

(20    0.21
 6     0.55
 26    0.23
 5     0.56
 22    0.40
 12    0.27
 18    0.19
 1     0.25
 9     0.61
 13    0.29
 11    0.28
 23    0.45
 15    0.55
 3     0.52
 25    0.54
 14    0.49
 2     0.47
 24    0.50
 19    0.18
 16    0.25
 21    0.26
 17    0.58
 0     0.22
 4     0.46
 Name: age, dtype: float64,
 8     0.62
 7     0.60
 10    0.18
 Name: age, dtype: float64)

In [8]:
import tensorflow as tf
import keras

In [9]:
model = keras.Sequential([
    keras.layers.Dense(1, input_shape=(1,), activation='sigmoid', kernel_initializer='ones', bias_initializer='zeros')
])

model.compile(
    optimizer="adam",
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [10]:
model.fit(X_train_scaled, y_train, epochs=700)

Epoch 1/700
1/1 [==============================] - 0s 485ms/step - loss: 0.6642 - accuracy: 0.5000
Epoch 2/700
1/1 [==============================] - 0s 3ms/step - loss: 0.6641 - accuracy: 0.5000
Epoch 3/700
1/1 [==============================] - 0s 3ms/step - loss: 0.6640 - accuracy: 0.5000
Epoch 4/700
1/1 [==============================] - 0s 2ms/step - loss: 0.6639 - accuracy: 0.5000
Epoch 5/700
1/1 [==============================] - 0s 3ms/step - loss: 0.6637 - accuracy: 0.5000
Epoch 6/700
1/1 [==============================] - 0s 2ms/step - loss: 0.6636 - accuracy: 0.5000
Epoch 7/700
1/1 [==============================] - 0s 3ms/step - loss: 0.6635 - accuracy: 0.5000
Epoch 8/700
1/1 [==============================] - 0s 4ms/step - loss: 0.6634 - accuracy: 0.5000
Epoch 9/700
1/1 [==============================] - 0s 3ms/step - loss: 0.6633 - accuracy: 0.5000
Epoch 10/700
1/1 [==============================] - 0s 3ms/step - loss: 0.6632 - accuracy: 0.5000
Epoch 11/700
1/1 [=========

In [11]:
model.evaluate(X_test_scaled, y_test)

1/1 [==============================] - 0s 121ms/step - loss: 0.4940 - accuracy: 1.0000


[0.4940451681613922, 1.0]

In [12]:
model.predict(X_test_scaled)

array([[0.6428451 ],
       [0.6342728 ],
       [0.44289857]], dtype=float32)

In [13]:
X_test_scaled

8     0.62
7     0.60
10    0.18
Name: age, dtype: float64

In [14]:
y_test

8     1
7     1
10    0
Name: bought_insurance, dtype: int64

<br><br>
## _So now we will get weights and see how the pred with sigmoid functon work.._ 😉

In [15]:
# y = mx + c

coef, intercept = model.get_weights()

In [16]:
coef, intercept

(array([[1.8571382]], dtype=float32), array([-0.56369144], dtype=float32))

In [17]:
import math 

def sigmoid(x):
    return 1/(1 + math.exp(-x))

In [18]:
def pred_func(age):
    w1 = coef
    wsum = w1*age + intercept
    return sigmoid(wsum)

In [19]:
# so in we can see the peredicted val of our model when X_test = 0.62, get 0.64.. in y_pred which means the person will buy the insurance
pred_func(0.62)

0.6428451160842816

In [20]:
# log loss or binary cross entropy

def log_loss(y_true, y_pred):
    epln = 1e-15
    y_pred_n = [max(i, epln) for i in y_pred]
    y_pred_n = [min(i, 1-epln) for i in y_pred_n]
    y_pred_n = np.array(y_pred_n)
    return -np.mean(y_true*np.log(y_pred_n) + (1-y_true) * np.log(1-y_pred_n))

In [23]:
def sigmoid_numpy(x):
    return 1/(1 + np.exp(-x))

sigmoid_numpy(np.array([12, 0, 1]))

array([0.99999386, 0.5       , 0.73105858])

In [103]:
def grad_dcnt(age, y_true, epochs, loss_thresh):
    w1 = 0.0001
    bias = 0
    l_rate = 0.1
    n = len(age)
    for i in range(epochs):
        wsum = w1*age + bias
        y_pred = sigmoid_numpy(wsum)
        
        loss = log_loss(y_true, y_pred)
        
        w1_d = (1/n)*np.dot(np.transpose(age), (y_pred-y_true))
        bias_d = np.mean((y_pred-y_true))
        
        w1 = w1*l_rate - w1_d
        bias = bias - l_rate*bias_d
        
        print(f"Epoch: {i}, w1: {w1}, bias: {bias}, loss: {loss}")
        
        if loss<=loss_thresh:
            break
        
    return w1, bias

In [106]:
grad_dcnt(X_train_scaled, y_train, 700, 0.61)

Epoch: 0, w1: 0.05021404572916755, bias: -9.697916664936727e-07, loss: 0.6931421599409923
Epoch: 1, w1: 0.0530769636225658, bias: -0.0004878953426459914, loss: 0.6906800658017004
Epoch: 2, w1: 0.05328773961789692, bias: -0.0009904095821301685, loss: 0.6905401858621644
Epoch: 3, w1: 0.053348507938129626, bias: -0.0014824061919909782, loss: 0.6905275698585357
Epoch: 4, w1: 0.05339968654104528, bias: -0.0019626934782514304, loss: 0.6905222586076606
Epoch: 5, w1: 0.053449182095646745, bias: -0.0024314711037885317, loss: 0.6905175184618496
Epoch: 6, w1: 0.05349746565736785, bias: -0.0028890104126644005, loss: 0.6905129650139464
Epoch: 7, w1: 0.05354459043000071, bias: -0.003335580538162623, loss: 0.6905085706882946
Epoch: 8, w1: 0.05359058553222275, bias: -0.0037714444059710204, loss: 0.6905043282012082
Epoch: 9, w1: 0.05363547810611131, bias: -0.0041968586554928, loss: 0.6905002316489277
Epoch: 10, w1: 0.053679294572490474, bias: -0.004612073777526095, loss: 0.6904962754348682
Epoch: 11, w

(0.05546312195740795, -0.021515069160035854)